# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [1]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU

# from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp

import os
import warnings

import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
import pytorch_lightning as pl

from data import DataModule, inject_synth_bias, load_credit, preprocess_credit
from metrics import eval_model
from models.DECAF import DECAF

warnings.filterwarnings("ignore")

/mnt/c/Users/dariu/Documents/debiasing_goggle/UvA_FACT2022/decaf/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/c/Users/dariu/Documents/debiasing_goggle/UvA_FACT2022/decaf/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/c/Users/dariu/Documents/debiasing_goggle/UvA_FACT2022/decaf/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# goggle evaluation
from synthcity.plugins.core.dataloader import GenericDataLoader
from synthcity.metrics import eval_detection, eval_performance, eval_statistical

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


## Loading data

In [3]:
dataset = load_adult()
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


Preprocess the data next in order to make it suitable for training models on.

In [4]:
dataset = preprocess_adult(dataset)
dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,0.833333,0.043350,0.000000,0.800000,0.333333,0.615385,0.6,0.0,1.0,0.02174,0.0,0.397959,0.0,1.0
1,0.452055,0.166667,0.047274,0.000000,0.800000,0.000000,0.307692,0.4,0.0,1.0,0.00000,0.0,0.122449,0.0,1.0
2,0.287671,0.000000,0.136877,0.200000,0.533333,0.166667,0.461538,0.6,0.0,1.0,0.00000,0.0,0.397959,0.0,1.0
3,0.493151,0.000000,0.149792,0.133333,0.400000,0.000000,0.461538,0.4,1.0,1.0,0.00000,0.0,0.397959,0.0,1.0
4,0.150685,0.000000,0.219998,0.000000,0.800000,0.000000,0.384615,0.0,1.0,0.0,0.00000,0.0,0.397959,0.3,1.0


Split the dataset into train and test folds. Test fold size is 2000.

In [5]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

Size of train set: 43222
Size of test set: 2000


### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [6]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [7]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [8]:
models_dir = './cache/'

def train_decaf(train_dataset, dag_seed, test_dataset, biased_edges={}, h_dim=200, lr=0.5e-3,
                batch_size=64, lambda_privacy=0, lambda_gp=10, d_updates=10,
                alpha=2, rho=2, weight_decay=1e-2, grad_dag_loss=False, l1_g=0,
                l1_W=1e-4, p_gen=-1, use_mask=True, epochs=50, model_name='decaf', generate_test=False):
    
    model_filename = os.path.join(models_dir, f'{model_name}.pkl')

    dm = DataModule(train_dataset.values)
    dm_test = DataModule(test_dataset.values)
     
    model = DECAF(
        dm.dims[0],
        dag_seed=dag_seed,
        h_dim=h_dim,
        lr=lr,
        batch_size=batch_size,
        lambda_privacy=lambda_privacy,
        lambda_gp=lambda_gp,
        d_updates=d_updates,
        alpha=alpha,
        rho=rho,
        weight_decay=weight_decay,
        grad_dag_loss=grad_dag_loss,
        l1_g=l1_g,
        l1_W=l1_W,
        p_gen=p_gen,
        use_mask=use_mask,
    )

    if os.path.exists(model_filename):
        model = torch.load(model_filename)
    else:
        trainer = pl.Trainer(max_epochs=epochs, logger=False)
        trainer.fit(model, dm)
        torch.save(model, model_filename)

    # Generate synthetic data
    synth_dataset = (
        model.gen_synthetic(
            dm.dataset.x,
            gen_order=model.get_gen_order(),
            biased_edges=biased_edges,
        )
        .detach()
        .numpy()
    )
    synth_dataset[:, -1] = synth_dataset[:, -1].astype(np.int8)

    synth_dataset = pd.DataFrame(synth_dataset,
                                 index=train_dataset.index,
                                 columns=train_dataset.columns)
    if generate_test:
        synth_dataset_x_test = (
            model.gen_synthetic(
                dm_test.dataset.x,
                gen_order=model.get_gen_order(),
                biased_edges=biased_edges,
            )
            .detach()
            .numpy()
        )
        synth_dataset_x_test[:, -1] = synth_dataset_x_test[:, -1].astype(np.int8)

        synth_dataset_x_test = pd.DataFrame(synth_dataset_x_test,
                                    index=test_dataset.index,
                                    columns=test_dataset.columns)

    if 'approved' in synth_dataset.columns:
        # Binarise columns for credit dataset
        synth_dataset['ethnicity'] = np.round(synth_dataset['ethnicity'])
        synth_dataset['approved'] = np.round(synth_dataset['approved'])
    else:
        # Binarise columns for adult dataset
        synth_dataset['sex'] = np.round(synth_dataset['sex'])
        synth_dataset['income'] = np.round(synth_dataset['income'])

        synth_dataset_x_test['sex'] = np.round(synth_dataset_x_test['sex'])
        synth_dataset_x_test['income'] = np.round(synth_dataset_x_test['income'])

    return synth_dataset, synth_dataset_x_test

In [8]:
def eval_model(dataset_train, dataset_test):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

In [9]:
def eval_goggle(X_test, X_synth):
    quality_evaluator = eval_statistical.AlphaPrecision()
    qual_res = quality_evaluator.evaluate(X_test, X_synth)
    qual_res = {
        k: v for (k, v) in qual_res.items() if "naive" in k
    }  # use the naive implementation of AlphaPrecision
    qual_score = np.mean(list(qual_res.values()))

    xgb_evaluator = eval_performance.PerformanceEvaluatorXGB()
    linear_evaluator = eval_performance.PerformanceEvaluatorLinear()
    mlp_evaluator = eval_performance.PerformanceEvaluatorMLP()
    xgb_score = xgb_evaluator.evaluate(X_test, X_synth)
    linear_score = linear_evaluator.evaluate(X_test, X_synth)
    mlp_score = mlp_evaluator.evaluate(X_test, X_synth)
    gt_perf = (xgb_score["gt"] + linear_score["gt"] + mlp_score["gt"]) / 3
    synth_perf = (
        xgb_score["syn_ood"] + linear_score["syn_ood"] + mlp_score["syn_ood"]
    ) / 3

    xgb_detector = eval_detection.SyntheticDetectionXGB()
    mlp_detector = eval_detection.SyntheticDetectionMLP()
    gmm_detector = eval_detection.SyntheticDetectionGMM()
    xgb_det = xgb_detector.evaluate(X_test, X_synth)
    mlp_det = mlp_detector.evaluate(X_test, X_synth)
    gmm_det = gmm_detector.evaluate(X_test, X_synth)
    det_score = (xgb_det["mean"] + mlp_det["mean"] + gmm_det["mean"]) / 3

    return qual_score, (gt_perf, synth_perf), det_score

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [11]:
eval_model(dataset_train, dataset_test)

{'precision': 0.8834394904458599,
 'recall': 0.9222074468085106,
 'auroc': 0.7766279169526424,
 'dp': 0.19207586973309976,
 'ftu': 0.02300000000000002}

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [31]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

2023-07-24 13:43:38.290828: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-24 13:43:38.557989: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-24 13:43:38.564195: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-24 13:43:38.573554: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-V7B5NBEP): /proc/driver/nvidia/version does not exist
2023-07-24 13:43:40.115972: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow 

Epoch 17:  40%|████      | 271/676 [116:04:05<173:27:34, 1541.86s/it, loss=-0.173]


  0%|          | 0/50 [00:00<?, ?it/s]2023-07-24 13:43:47.621733: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-07-24 13:43:47.655306: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2419205000 Hz


0 [D loss: 0.000159, acc.: 100.00%] [G loss: 40.029396]


  2%|▏         | 1/50 [00:27<22:31, 27.58s/it]

...

48 [D loss: 0.000000, acc.: 100.00%] [G loss: 356.410614]


100%|██████████| 50/50 [16:36<00:00, 19.94s/it]


49 [D loss: 0.000000, acc.: 100.00%] [G loss: 381.815826]


Synthetic data generation: 100%|██████████| 338/338 [00:06<00:00, 53.68it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.000553,0.666667,0.014988,0.333333,0.133333,0.333333,0.846154,0.6,0.00,1.0,0.0261,-0.034176,-0.017084,0.425,0.0
1,-0.000553,0.333333,0.014988,0.666667,0.533333,0.333333,0.846154,1.0,0.75,1.0,0.0261,-0.034176,-0.017084,0.275,1.0
2,-0.000553,0.833333,0.014988,0.066667,0.933333,0.666667,0.000000,0.2,0.25,1.0,0.0261,-0.034176,-0.017084,0.150,1.0
3,-0.000553,0.166667,0.014988,0.466667,0.800000,0.166667,0.230769,0.4,0.00,0.0,0.0261,-0.034176,-0.017084,0.350,1.0
4,-0.000553,0.833333,0.014988,0.400000,0.533333,0.500000,0.923077,0.0,0.50,0.0,0.0261,-0.034176,-0.017084,0.475,1.0


In [10]:
eval_model(synth_data, dataset_test)

{'precision': 0.6971736204576043,
 'recall': 0.6888297872340425,
 'auroc': 0.39078586135895677,
 'dp': 0.7117996939976097,
 'ftu': 0.47}

### WGAN-GP

In [11]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

  2%|▏         | 1/50 [00:05<04:48,  5.88s/it]

Epoch: 0 | disc_loss: 0.15245389938354492 | gen_loss: -0.06989498436450958


  4%|▍         | 2/50 [00:10<03:54,  4.89s/it]

Epoch: 1 | disc_loss: 0.0009180586785078049 | gen_loss: -0.0033318146597594023


  6%|▌         | 3/50 [00:14<03:36,  4.60s/it]

...

Epoch: 48 | disc_loss: -0.08710119128227234 | gen_loss: 0.041870132088661194


100%|██████████| 50/50 [03:51<00:00,  4.62s/it]


Epoch: 49 | disc_loss: -0.10412894934415817 | gen_loss: 0.05354977026581764


Synthetic data generation: 100%|██████████| 87/87 [00:01<00:00, 69.38it/s]


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.415160,0.333333,0.136200,0.333333,0.600000,0.166667,0.307692,1.0,1.00,1.0,0.000021,0.041940,0.407321,0.375,0.0
1,0.452994,0.666667,0.128935,0.400000,0.800000,0.500000,0.846154,0.2,0.75,0.0,-0.026776,0.029268,0.403623,0.100,0.0
2,0.500516,0.166667,0.204948,0.066667,0.933333,0.666667,1.000000,0.0,0.00,1.0,-0.028636,0.001469,0.480637,0.400,0.0
3,0.208567,0.500000,0.118594,0.600000,0.800000,0.833333,0.307692,0.0,1.00,0.0,-0.027901,0.015777,0.280495,0.475,0.0
4,0.430072,0.166667,0.126853,0.666667,0.466667,0.000000,0.384615,0.0,0.00,0.0,-0.030149,0.020552,0.380429,0.650,1.0


In [12]:
eval_model(synth_data, dataset_test)

{'precision': 0.7207890743550834,
 'recall': 0.3158244680851064,
 'auroc': 0.47242836307481123,
 'dp': 0.10060642419000054,
 'ftu': 0.14}

### FairGAN

In [13]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
cache/adult.npy


2022-02-04 17:11:43.219249: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


Pretrain_Epoch:0, trainLoss:0.018377, validLoss:0.007400, validReverseLoss:0.000000
Pretrain_Epoch:1, trainLoss:0.007770, validLoss:0.007489, validReverseLoss:0.000000
Pretrain_Epoch:2, trainLoss:0.007857, validLoss:0.007542, validReverseLoss:0.000000
Pretrain_Epoch:3, trainLoss:0.007895, validLoss:0.007547, validReverseLoss:0.000000
Pretrain_Epoch:4, trainLoss:0.007891, validLoss:0.006571, validReverseLoss:0.000000
Pretrain_Epoch:5, trainLoss:0.002940, validLoss:0.002490, validReverseLoss:0.000000
Pretrain_Epoch:6, trainLoss:0.002588, validLoss:0.002484, validReverseLoss:0.000000
Pretrain_Epoch:7, trainLoss:0.002579, validLoss:0.002462, validReverseLoss:0.000000
Pretrain_Epoch:8, trainLoss:0.002581, validLoss:0.002461, validReverseLoss:0.000000
Pretrain_Epoch:9, trainLoss:0.002567, validLoss:0.002443, validReverseLoss:0.000000
Pretrain_Epoch:10, trainLoss:0.002573, validLoss:0.002476, validReverseLoss:0.000000
Pretrain_Epoch:11, trainLoss:0.002570, validLoss:0.002426, validReverseLoss

/Users/*****/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/*****/Projects/UvA/UvA_FACT2022/env/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.000000,0.313098,0.000000,0.447853,0.307014,0.633629,0.370685,0.0,0.414991,0.0,0.000000,0.369372,0.526167,1.048100,1.0
1,0.162797,0.153582,0.032848,0.780304,0.193229,0.461950,0.398713,0.0,0.813397,0.0,0.000000,0.385241,0.000000,1.102639,1.0
2,0.000000,0.208462,0.239274,0.839121,0.000000,0.545892,0.406260,0.0,1.180958,0.0,0.000000,0.410443,0.000000,0.000000,1.0
3,0.149550,0.366827,0.022638,0.792689,0.087440,0.244400,0.490716,0.0,0.860048,0.0,0.000000,0.332419,0.000000,0.363599,1.0
4,0.063530,0.319478,0.718697,0.598897,0.166194,0.573069,0.628330,0.0,1.016383,0.0,0.070529,0.514503,0.000000,0.772208,1.0


In [14]:
eval_model(synth_data, dataset_test)

{'precision': 0.773604590505999,
 'recall': 0.9860372340425532,
 'auroc': 0.5555186170212766,
 'dp': 0.01400030693248877,
 'ftu': 0.030999999999999917}

### DECAF

#### DECAF-ND

In [13]:
_, synth_data= train_decaf(dataset_train, dag_seed, dataset_test, generate_test=True)
synth_data.head()

# save synthetic sample
dataset_test.to_csv('x_test_adult.csv')
synth_data.to_csv('x_synth_adult.csv')


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

In [21]:
eval_model(synth_data, dataset_test)

{'precision': 0.8819599109131403,
 'recall': 0.7898936170212766,
 'auroc': 0.7346645504461221,
 'dp': 0.32534373,
 'ftu': 0.099999964}

In [10]:
# load synthetic and test
X_test = pd.read_csv('x_test_adult.csv', index_col=0)
X_synth_test = pd.read_csv('x_synth_adult.csv', index_col=0)

In [11]:
label = "income"

X_synth_loader = GenericDataLoader(
    X_synth_test,
    target_column=label,
)
X_test_loader = GenericDataLoader(
    X_test,
    target_column=label,
)

res = eval_goggle(X_synth_loader, X_test_loader)

print(f"Quality: {res[0]:.3f}")
print(f"Detection: {res[2]:.3f}")
print(
f"Performance on real: {res[1][0]:.3f}, on synth: {res[1][1]:.3f}, diff: {(res[1][0] - res[1][1]):.3f}")

Quality: 0.572
Detection: 0.698
Performance on real: 0.718, on synth: 0.640, diff: 0.078


#### DECAF-FTU

In [12]:
_, synth_data = train_decaf(dataset_train, dag_seed, dataset_test, biased_edges=bias_dict_ftu, generate_test=True)
synth_data.head()

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
11086,0.495013,1.072265e-07,0.035992,0.269277,0.502386,0.001513,0.066416,0.366714,1.881836e-30,1.0,0.000006,1.258836e-13,0.566528,1.836281e-06,0.0
13040,0.494979,9.606715e-06,0.067790,0.170352,0.508076,0.426382,0.460906,0.980978,9.967096e-01,0.0,0.010387,4.444226e-02,0.338800,2.679395e-05,0.0
12915,0.148479,3.297513e-07,0.031570,0.274266,0.811098,0.406156,0.095585,0.312192,9.700151e-01,1.0,0.000444,7.830165e-11,0.468205,1.945665e-11,1.0
8828,0.132333,5.430275e-01,0.041796,0.602310,0.509497,0.387588,0.148159,0.828860,4.584910e-29,1.0,0.000146,3.218246e-07,0.306507,2.295997e-03,1.0
12594,0.170732,8.479881e-03,0.157833,0.208199,0.650439,0.162866,0.227585,0.257204,1.044550e-05,1.0,0.000869,6.504069e-13,0.297163,2.777788e-17,1.0


In [13]:
# save synthetic sample
dataset_test.to_csv('x_test_adult_ftu.csv')
synth_data.to_csv('x_synth_adult_ftu.csv')

In [23]:
eval_model(synth_data, dataset_test)

{'precision': 0.8622412562455389,
 'recall': 0.8031914893617021,
 'auroc': 0.7070392930679478,
 'dp': 0.2167862,
 'ftu': 0.0015000105}

In [12]:
# load synthetic and test
X_test = pd.read_csv('x_test_adult_ftu.csv', index_col=0)
X_synth_test = pd.read_csv('x_synth_adult_ftu.csv', index_col=0)

In [13]:
label = "income"

X_synth_loader = GenericDataLoader(
    X_synth_test,
    target_column=label,
)
X_test_loader = GenericDataLoader(
    X_test,
    target_column=label,
)

res = eval_goggle(X_synth_loader, X_test_loader)

print(f"Quality: {res[0]:.3f}")
print(f"Detection: {res[2]:.3f}")
print(
f"Performance on real: {res[1][0]:.3f}, on synth: {res[1][1]:.3f}, diff: {(res[1][0] - res[1][1]):.3f}")

Quality: 0.567
Detection: 0.698
Performance on real: 0.677, on synth: 0.671, diff: 0.006


#### DECAF-CF

In [14]:
_, synth_data = train_decaf(dataset_train, dag_seed, dataset_test, biased_edges=bias_dict_cf, generate_test=True)
synth_data.head()

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
11086,0.494885,1.496933e-05,0.037080,0.068646,0.509228,0.000149,0.419497,0.393270,3.492381e-15,1.0,1.011338e-07,5.851650e-15,0.566989,2.633086e-19,0.0
13040,0.494936,4.529162e-05,0.035780,0.131511,0.692952,0.309115,0.464737,0.440604,2.211955e-04,1.0,7.582552e-07,1.849083e-06,0.260473,1.530963e-12,1.0
12915,0.070165,1.562485e-03,0.230961,0.169594,0.580689,0.194124,0.039334,0.278739,5.385244e-37,1.0,1.844534e-05,1.888660e-12,0.233035,4.324961e-05,1.0
8828,0.494861,5.945465e-07,0.028139,0.095030,0.499025,0.003266,0.117451,0.449909,5.955176e-28,1.0,1.204840e-03,3.345086e-16,0.572020,1.913983e-20,0.0
12594,0.494926,7.365664e-01,0.135249,0.711014,0.771396,0.000006,0.343281,0.350653,8.121139e-31,1.0,2.099754e-07,2.198741e-06,0.420037,5.059750e-11,1.0


In [15]:
# save synthetic sample
dataset_test.to_csv('x_test_adult_cf.csv')
synth_data.to_csv('x_synth_adult_cf.csv')

In [20]:
eval_model(synth_data, dataset_test)

{'precision': 0.7692307692307693,
 'recall': 0.9441489361702128,
 'auroc': 0.5426389842141386,
 'dp': 0.0018555522,
 'ftu': 0.07699996}

In [14]:
# load synthetic and test
X_test = pd.read_csv('x_test_adult_cf.csv', index_col=0)
X_synth_test = pd.read_csv('x_synth_adult_cf.csv', index_col=0)

In [15]:
label = "income"

X_synth_loader = GenericDataLoader(
    X_synth_test,
    target_column=label,
)
X_test_loader = GenericDataLoader(
    X_test,
    target_column=label,
)

res = eval_goggle(X_synth_loader, X_test_loader)

print(f"Quality: {res[0]:.3f}")
print(f"Detection: {res[2]:.3f}")
print(
f"Performance on real: {res[1][0]:.3f}, on synth: {res[1][1]:.3f}, diff: {(res[1][0] - res[1][1]):.3f}")

Quality: 0.571
Detection: 0.708
Performance on real: 0.549, on synth: 0.551, diff: -0.002


#### DECAF-DP

In [9]:
_, synth_data = train_decaf(dataset_train, dag_seed, dataset_test, biased_edges=bias_dict_dp, generate_test=True)
synth_data.head()

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
10434,0.165051,0.004011,0.233374,0.084554,0.509286,0.000177,0.219202,0.338384,1.763635e-27,1.0,6.160838e-06,3.714814e-02,0.403032,2.843400e-18,1.0
7471,0.442097,0.301965,0.038136,0.148809,0.509486,0.009816,0.103833,0.387248,1.813753e-37,1.0,1.659088e-10,8.341237e-16,0.294574,3.544383e-18,0.0
4377,0.493849,0.000076,0.039821,0.130236,0.508173,0.004529,0.700785,0.405840,6.146480e-23,1.0,1.918529e-06,1.647496e-10,0.327960,8.761260e-11,0.0
30452,0.241193,0.016223,0.174061,0.886941,0.699391,0.371640,0.116590,0.771863,8.329799e-01,1.0,8.071691e-06,4.995499e-12,0.536204,2.946659e-04,1.0
9045,0.494885,0.741972,0.077021,0.199371,0.660638,0.000086,0.832221,0.372516,6.276626e-01,1.0,7.832788e-07,3.580257e-03,0.269290,2.492106e-15,1.0


In [10]:
# save synthetic sample
dataset_test.to_csv('x_test_adult_dp.csv')
synth_data.to_csv('x_synth_adult_dp.csv')

In [22]:
# decaf evaluation
eval_model(synth_data, dataset_test)

{'precision': 0.7696105320899616,
 'recall': 0.932845744680851,
 'auroc': 0.543035775566232,
 'dp': 0.045112073,
 'ftu': 0.021500051}

In [16]:
# load synthetic and test
X_test = pd.read_csv('x_test_adult_dp.csv', index_col=0)
X_synth_test = pd.read_csv('x_synth_adult_dp.csv', index_col=0)

In [17]:
label = "income"

X_synth_loader = GenericDataLoader(
    X_synth_test,
    target_column=label,
)
X_test_loader = GenericDataLoader(
    X_test,
    target_column=label,
)

res = eval_goggle(X_synth_loader, X_test_loader)

print(f"Quality: {res[0]:.3f}")
print(f"Detection: {res[2]:.3f}")
print(
f"Performance on real: {res[1][0]:.3f}, on synth: {res[1][1]:.3f}, diff: {(res[1][0] - res[1][1]):.3f}")

Quality: 0.560
Detection: 0.696
Performance on real: 0.555, on synth: 0.508, diff: 0.047
